In [ ]:
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from keras.optimizers import Adam

In [ ]:
def load_data_from_mat(file_path, key):
    data = loadmat(file_path)
    return data[key]


In [ ]:
# Define paths and keys for each class
file_info_dict = {
    'A': [('/content/drive/MyDrive/emdvmd/VMDF.mat', 'IMFvmd1F')],
    'B': [('/content/drive/MyDrive/emdvmd/VMDN.mat', 'IMFvmd1N')],
    'C': [('/content/drive/MyDrive/emdvmd/VMDO.mat', 'IMFvmd1O')],
    'D': [('/content/drive/MyDrive/emdvmd/VMDS.mat', 'IMFvmd1S')],
    'E': [('/content/drive/MyDrive/emdvmd/VMDZ.mat', 'IMFvmd1Z')]
}

In [ ]:
def prepare_data_for_classification(classes):
    X, y = [], []
    for label, class_id in enumerate(classes):
        file_info = file_info_dict[class_id]  # Get file paths and keys for each class
        for file_path, key in file_info:
            data = load_data_from_mat(file_path, key)
            X.append(data)
            y.append(np.array([label] * data.shape[0]))  # Assign the same label to all samples
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y


In [ ]:
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(units=32))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, recall, f1

In [ ]:
def train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, input_shape, num_classes):
 # Create the CNN-LSTM model
    model = create_cnn_lstm_model(input_shape, num_classes)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

    # Predict classes
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Compute additional metrics
    accuracy, recall, f1 = compute_metrics(y_test, y_pred_classes)

    # Print the metrics
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

    return model, accuracy, recall, f1

In [ ]:
# Example: AB vs CDE Classification
classes = ['A', 'B', 'C', 'D', 'E']  # Use the appropriate subset of classes for each task
X, y = prepare_data_for_classification(classes)

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get input shape and number of classes
input_shape = (X_train.shape[1], 1)
num_classes = len(np.unique(y))

# Reshape X to fit the CNN-LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train and evaluate
model, accuracy, recall, f1 = train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, input_shape, num_classes)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 40s 66ms/step - accuracy: 0.5542 - loss: 0.9581 - val_accuracy: 0.7169 - val_loss: 0.6003
Epoch 2/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.7304 - loss: 0.5709 - val_accuracy: 0.7769 - val_loss: 0.4436
Epoch 3/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 44s 66ms/step - accuracy: 0.7536 - loss: 0.4950 - val_accuracy: 0.7952 - val_loss: 0.4185
Epoch 4/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.7703 - loss: 0.4556 - val_accuracy: 0.8223 - val_loss: 0.3937
Epoch 5/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.8016 - loss: 0.4407 - val_accuracy: 0.8221 - val_loss: 0.3823
Epoch 6/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 32s 62ms/step - accuracy: 0.8183 - loss: 0.3931 - val_accuracy: 0.8245 - val_loss: 0.4119
Epoch 7/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 31s 60ms/step - accuracy: 0.8197 - loss: 0.3955 - val_accuracy: 0.8462 - val_loss: 0.3494
Epoch 8/50
513/513 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.8218 - loss: 0.3785 - 

In [ ]:
# AE Classification
classes_ae = ['A', 'E']

# Prepare data
X_ae, y_ae = prepare_data_for_classification(classes_ae)

# Merge and create new labels
X = X_ae
y = y_ae

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get input shape and number of classes
input_shape = (X_train.shape[1], 1)
num_classes = len(np.unique(y))

# Reshape X to fit the CNN-LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train and evaluate
model, accuracy, recall, f1 = train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, input_shape, num_classes)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 33s 114ms/step - accuracy: 0.7086 - loss: 0.5395 - val_accuracy: 0.8554 - val_loss: 0.3246
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 30s 63ms/step - accuracy: 0.8882 - loss: 0.2899 - val_accuracy: 0.8658 - val_loss: 0.2967
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 13s 62ms/step - accuracy: 0.9318 - loss: 0.1951 - val_accuracy: 0.9323 - val_loss: 0.1590
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step - accuracy: 0.9508 - loss: 0.1320 - val_accuracy: 0.9152 - val_loss: 0.2145
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - accuracy: 0.9594 - loss: 0.1108 - val_accuracy: 0.9707 - val_loss: 0.0827
Epoch 6/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - accuracy: 0.9553 - loss: 0.1179 - val_accuracy: 0.9549 - val_loss: 0.1278
Epoch 7/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 59ms/step - accuracy: 0.9583 - loss: 0.1190 - val_accuracy: 0.9719 - val_loss: 0.0744
Epoch 8/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - accuracy: 0.9660 - loss: 0.0954 -

In [ ]:
# CE Classification
classes_ce = ['C', 'E']

# Prepare data
X_ce, y_ce = prepare_data_for_classification(classes_ce)

# Merge and create new labels
X = X_ce
y = y_ce

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get input shape and number of classes
input_shape = (X_train.shape[1], 1)
num_classes = len(np.unique(y))

# Reshape X to fit the CNN-LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train and evaluate
model, accuracy, recall, f1 = train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, input_shape, num_classes)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 17s 60ms/step - accuracy: 0.6922 - loss: 0.5085 - val_accuracy: 0.9378 - val_loss: 0.1889
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - accuracy: 0.9552 - loss: 0.1258 - val_accuracy: 0.9426 - val_loss: 0.1442
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 23s 63ms/step - accuracy: 0.9569 - loss: 0.1214 - val_accuracy: 0.9591 - val_loss: 0.0958
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 13s 63ms/step - accuracy: 0.9589 - loss: 0.1223 - val_accuracy: 0.9634 - val_loss: 0.0967
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step - accuracy: 0.9625 - loss: 0.1121 - val_accuracy: 0.9622 - val_loss: 0.1149
Epoch 6/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 68ms/step - accuracy: 0.9564 - loss: 0.1268 - val_accuracy: 0.9622 - val_loss: 0.0918
Epoch 7/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 15s 73ms/step - accuracy: 0.9677 - loss: 0.0985 - val_accuracy: 0.9689 - val_loss: 0.0719
Epoch 8/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.9650 - loss: 0.0960 - 

In [ ]:
# BE Classification
classes_be = ['B', 'E']

# Prepare data
X_be, y_be = prepare_data_for_classification(classes_be)

# Merge and create new labels
X = X_be
y = y_be

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get input shape and number of classes
input_shape = (X_train.shape[1], 1)
num_classes = len(np.unique(y))

# Reshape X to fit the CNN-LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train and evaluate
model, accuracy, recall, f1 = train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, input_shape, num_classes)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 19s 64ms/step - accuracy: 0.7457 - loss: 0.4592 - val_accuracy: 0.9500 - val_loss: 0.1349
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 20s 60ms/step - accuracy: 0.9368 - loss: 0.1733 - val_accuracy: 0.9487 - val_loss: 0.1614
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 19s 53ms/step - accuracy: 0.9396 - loss: 0.1794 - val_accuracy: 0.9262 - val_loss: 0.1911
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - accuracy: 0.9518 - loss: 0.1450 - val_accuracy: 0.9683 - val_loss: 0.1030
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step - accuracy: 0.9512 - loss: 0.1442 - val_accuracy: 0.9689 - val_loss: 0.0971
Epoch 6/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step - accuracy: 0.9636 - loss: 0.1154 - val_accuracy: 0.9677 - val_loss: 0.0955
Epoch 7/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - accuracy: 0.9560 - loss: 0.1249 - val_accuracy: 0.9518 - val_loss: 0.1210
Epoch 8/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 20s 56ms/step - accuracy: 0.9589 - loss: 0.1177 - 

In [ ]:
# BD Classification
classes_bd = ['B', 'D']

# Prepare data
X_bd, y_bd = prepare_data_for_classification(classes_bd)

# Merge and create new labels
X = X_bd
y = y_bd

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get input shape and number of classes
input_shape = (X_train.shape[1], 1)
num_classes = len(np.unique(y))

# Reshape X to fit the CNN-LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train and evaluate
model, accuracy, recall, f1 = train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, input_shape, num_classes)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - accuracy: 0.8818 - loss: 0.1871 - val_accuracy: 0.9963 - val_loss: 0.0247
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - accuracy: 0.9882 - loss: 0.0282 - val_accuracy: 1.0000 - val_loss: 5.4930e-04
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 19s 55ms/step - accuracy: 0.9999 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 2.8415e-04
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 21s 60ms/step - accuracy: 0.9996 - loss: 0.0026 - val_accuracy: 0.9988 - val_loss: 0.0087
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 19s 53ms/step - accuracy: 0.9992 - loss: 0.0052 - val_accuracy: 1.0000 - val_loss: 4.2928e-04
Epoch 6/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - accuracy: 0.9999 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 5.1330e-04
Epoch 7/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step - accuracy: 0.9988 - loss: 0.0034 - val_accuracy: 0.9988 - val_loss: 0.0111
Epoch 8/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.9994 -

In [ ]:
# AC Classification
classes_ac = ['A', 'C']

# Prepare data
X_ac, y_ac = prepare_data_for_classification(classes_ac)

# Merge and create new labels
X = X_ac
y = y_ac

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get input shape and number of classes
input_shape = (X_train.shape[1], 1)
num_classes = len(np.unique(y))

# Reshape X to fit the CNN-LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train and evaluate
model, accuracy, recall, f1 = train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, input_shape, num_classes)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - accuracy: 0.7856 - loss: 0.3819 - val_accuracy: 0.9494 - val_loss: 0.1511
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 63ms/step - accuracy: 0.9779 - loss: 0.0778 - val_accuracy: 0.9915 - val_loss: 0.0315
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.9914 - loss: 0.0334 - val_accuracy: 0.9939 - val_loss: 0.0262
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 19s 58ms/step - accuracy: 0.9879 - loss: 0.0443 - val_accuracy: 0.9878 - val_loss: 0.0385
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.9919 - loss: 0.0325 - val_accuracy: 0.9951 - val_loss: 0.0257
Epoch 6/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 58ms/step - accuracy: 0.9963 - loss: 0.0165 - val_accuracy: 0.9957 - val_loss: 0.0201
Epoch 7/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.9949 - loss: 0.0176 - val_accuracy: 0.9957 - val_loss: 0.0217
Epoch 8/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 59ms/step - accuracy: 0.9927 - loss: 0.0243 - val_accurac